In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

import pickle

from statsmodels.tsa.stattools import adfuller,kpss
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR

In [ ]:
%cd drive/MyDrive/ime_project/
!ls

/content/drive/MyDrive/ime_project
capston_EDA.ipynb  EDA_1003.ipynb  EDA_1101.ipynb	     feature_selection.ipynb
data		   EDA_1010.ipynb  EDA_1102.ipynb	     result
DL		   EDA_1019.ipynb  EDA_1201.ipynb	     trial.ipynb
done		   EDA_1030.ipynb  feature_selection2.ipynb  var.ipynb


In [ ]:
df = pd.read_csv("result/dataset.csv", index_col="Unnamed: 0")
df.index = pd.to_datetime(df.index)
print(len(df.columns))
df.head()

86


,Ahu1_inv,Ahu2_inv,Ahu3_inv,Ahu4_inv,Ahu5_inv,Ahu6_inv,Ahu7_inv,Ahu8_inv,Ahu9_inv,Ahu10_inv,...,Plant9_pm10,Plant9_temperature,Plant9_humidity,Plant10_air,Plant10_monoxide,Plant10_dioxide,Plant10_pm2d5,Plant10_pm10,Plant10_temperature,Plant10_humidity
2023-01-01 08:00:00,35.0,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,23.09,51.27,54.0,0.0,441.0,39.0,44.0,23.36,50.24
2023-01-01 08:10:00,35.0,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.0,23.49,50.67,94.0,1.0,480.0,37.0,41.0,23.73,50.04
2023-01-01 08:20:00,35.0,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.0,23.60,50.23,275.0,1.0,462.0,38.0,43.0,23.91,49.27
2023-01-01 08:30:00,35.0,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.0,23.76,49.85,0.0,0.0,458.0,37.0,41.0,23.99,48.96
2023-01-01 08:40:00,35.0,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26.0,23.94,48.34,0.0,1.0,418.0,26.0,29.0,24.16,47.43


# 정상성 평가

In [ ]:
data = [df[[i]] for i in df.columns]
name = df.columns.tolist()

stat_data = []
stat_name = []
# non-stationary
nstat_data = []
nstat_name = []
diff1_data = []
diff1_name = []
diff2_data = []
diff2_name = []

# print(len(data),len(name))

for index,var in enumerate(data):
# ADF 검정 수행
    result = kpss(var.dropna())
    adf_statistic = result[0]
    p_value = result[1]

    # print(f'ADF 통계량: {adf_statistic}')
    # print(f'p-value: {p_value}#dfddfdfdf')

    if p_value <= 0.05:
        print(f'KPSS 검정결과 {name[index]} 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.')

        # log = np.log1p(var)
        # log_first = pd.Series(var).diff().dropna()
        log_first = np.log1p(var.dropna()).diff().dropna()

        result = kpss(log_first)
        adf_statistic = result[0]
        p_value = result[1]

        # print(f'>> ADF 통계량: {adf_statistic}')
        # print(f'>> p-value: {p_value}')

        if p_value <= 0.05:
            print(f'>> KPSS 검정결과 1차분한 {name[index]} 시계열 데이터가 정상성을 가지지 않습니다.')

            log_second = np.log1p(var).diff().diff().dropna()

            result = kpss(log_second)
            adf_statistic = result[0]
            p_value = result[1]

            # print(f'>>>> ADF 통계량: {adf_statistic}')
            # print(f'>>>> p-value: {p_value}')

            if p_value <=0.05:
                print(f'>>>> KPSS 검정결과 2차분한 {name[index]} 시계열 데이터가 정상성을 가지지 않습니다.')
                nstat_data.append(var)
                nstat_name.append(name[index])

            else:
                print(f'>>>> KPSS 검정결과 2차분한 {name[index]} 시계열 데이터가 정상성을 가집니다.')
                diff2_data.append(var)
                diff2_name.append(name[index])

        else:
            print(f'>>>> KPSS 검정결과 1차분한 {name[index]} 시계열 데이터가 정상성을 가집니다.')
            diff1_data.append(var)
            diff1_name.append(name[index])

    else:
        print(f'ADF 검정결과 {name[index]} 시계열 데이터가 정상성을 가집니다.')
        stat_data.append(var)
        stat_name.append(name[index])


    print()

print("정상성 :", stat_name)
print("1차분결과 정상성: ", diff1_name)
print("2차분결과 정상성: ", diff2_name)
print('정상성 안띰 :', nstat_name)

KPSS 검정결과 Ahu1_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu1_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu2_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu2_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu3_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu3_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu4_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu4_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu5_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu5_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu6_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu6_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu7_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.


<ipython-input-4-69b502ac522f>:18: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  result = kpss(var.dropna())
<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(log_first)
<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(log_first)
<ipython-input-4-69b502ac522f>:18: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  result = kpss(var.dropna())
<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is

>>>> KPSS 검정결과 1차분한 Ahu7_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu8_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu8_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu9_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu9_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu10_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu10_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu11_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu11_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu12_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu12_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu13_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu13_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Ahu14_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Ahu14_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Chiller1_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Chiller1_inv 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Chiller2_inv 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Chiller2_inv 시계열 데이터가 정상성을 가집니다

<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(log_first)
<ipython-input-4-69b502ac522f>:18: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  result = kpss(var.dropna())
<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(log_first)
<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(log_first)
<ipython-input-4-69b502ac522f>:18: InterpolationWarning: The test statistic is ou

>>>> KPSS 검정결과 1차분한 Plant1_dioxide 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant1_pm2d5 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant1_pm2d5 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant1_pm10 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant1_pm10 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant1_temperature 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant1_temperature 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant1_humidity 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant1_humidity 시계열 데이터가 정상성을 가집니다.

ADF 검정결과 Plant2_air 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant2_monoxide 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant2_monoxide 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant2_dioxide 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant2_dioxide 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant2_pm2d5 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant2_pm2d5 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant2_pm10 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant2_pm10 시계열 데이터가 정상성

...

<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(log_first)
<ipython-input-4-69b502ac522f>:18: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  result = kpss(var.dropna())
<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(log_first)
<ipython-input-4-69b502ac522f>:18: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  result = kpss(var.dropna())
<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is

>>>> KPSS 검정결과 1차분한 Plant9_air 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant9_monoxide 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant9_monoxide 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant9_dioxide 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant9_dioxide 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant9_pm2d5 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant9_pm2d5 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant9_pm10 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant9_pm10 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant9_temperature 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant9_temperature 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant9_humidity 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant9_humidity 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant10_air 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant10_air 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant10_monoxide 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant10_monoxide 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant10_dioxide 시계

<ipython-input-4-69b502ac522f>:18: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  result = kpss(var.dropna())
<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(log_first)
<ipython-input-4-69b502ac522f>:18: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  result = kpss(var.dropna())
<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(log_first)
<ipython-input-4-69b502ac522f>:18: InterpolationWarning: The test statistic is

>>>> KPSS 검정결과 1차분한 Plant10_dioxide 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant10_pm2d5 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant10_pm2d5 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant10_pm10 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant10_pm10 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant10_temperature 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant10_temperature 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 Plant10_humidity 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 Plant10_humidity 시계열 데이터가 정상성을 가집니다.

정상성 : ['Plant2_air', 'Plant3_air', 'Plant3_dioxide', 'Plant5_air', 'Plant6_air', 'Plant7_air']
1차분결과 정상성:  ['Ahu1_inv', 'Ahu2_inv', 'Ahu3_inv', 'Ahu4_inv', 'Ahu5_inv', 'Ahu6_inv', 'Ahu7_inv', 'Ahu8_inv', 'Ahu9_inv', 'Ahu10_inv', 'Ahu11_inv', 'Ahu12_inv', 'Ahu13_inv', 'Ahu14_inv', 'Chiller1_inv', 'Chiller2_inv', 'Plant1_air', 'Plant1_monoxide', 'Plant1_dioxide', 'Plant1_pm2d5', 'Plant1_pm10', 'Plant1_temperature', 'Plant1_humidity', 'Plant2_monoxide', 'Plant2_dioxide', 'P

<ipython-input-4-69b502ac522f>:18: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  result = kpss(var.dropna())
<ipython-input-4-69b502ac522f>:32: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(log_first)


In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# pd.set_option('display.max_rows', None)  # 모든 행을 보이게 설정
# pd.set_option('display.max_columns', None)  # 모든 열을 보이게 설정
# pd.set_option('display.width', None)  # 셀의 너비를 최대로 설정
# pd.set_option('display.max_colwidth', None)  # 열의 최대 너비를 제한 없애기

kpss_result = pd.DataFrame()
kpss_index = df.columns
kpss_result.index = kpss_index
kpss_result['정상성'] = ['O' if i in stat_name else "X" for i in kpss_index]
kpss_result['1차분 정상성'] = ['O' if i in diff1_name else "X" for i in kpss_index]
# kpss_result['2차분 정상성'] = ['O' if i in diff2_name else "X" for i in kpss_index]
kpss_result['정상성 안띰'] = ['O' if i in nstat_name else "X" for i in kpss_index]
kpss_result

def replace_after_o(row):
    o_found = False
    for i in range(len(row)):
        if row[i] == 'O':
            o_found = True
        elif o_found and row[i] == 'X':
            row[i] = '-'
    return row

# Apply the function to each row of the dataframe
kpss_result = kpss_result.apply(replace_after_o, axis=1)
kpss_result

,정상성,1차분 정상성,정상성 안띰
Ahu1_inv,X,O,-
Ahu2_inv,X,O,-
Ahu3_inv,X,O,-
Ahu4_inv,X,O,-
Ahu5_inv,X,O,-
...,...,...,...
Plant10_dioxide,X,O,-
Plant10_pm2d5,X,O,-
Plant10_pm10,X,O,-
Plant10_temperature,X,O,-


In [ ]:
print("정상성 :", stat_name)
print("1차분결과 정상성: ", diff1_name)

정상성 : ['Plant2_air', 'Plant3_air', 'Plant3_dioxide', 'Plant5_air', 'Plant6_air', 'Plant7_air']
1차분결과 정상성:  ['Ahu1_inv', 'Ahu2_inv', 'Ahu3_inv', 'Ahu4_inv', 'Ahu5_inv', 'Ahu6_inv', 'Ahu7_inv', 'Ahu8_inv', 'Ahu9_inv', 'Ahu10_inv', 'Ahu11_inv', 'Ahu12_inv', 'Ahu13_inv', 'Ahu14_inv', 'Chiller1_inv', 'Chiller2_inv', 'Plant1_air', 'Plant1_monoxide', 'Plant1_dioxide', 'Plant1_pm2d5', 'Plant1_pm10', 'Plant1_temperature', 'Plant1_humidity', 'Plant2_monoxide', 'Plant2_dioxide', 'Plant2_pm2d5', 'Plant2_pm10', 'Plant2_temperature', 'Plant2_humidity', 'Plant3_monoxide', 'Plant3_pm2d5', 'Plant3_pm10', 'Plant3_temperature', 'Plant3_humidity', 'Plant4_air', 'Plant4_monoxide', 'Plant4_dioxide', 'Plant4_pm2d5', 'Plant4_pm10', 'Plant4_temperature', 'Plant4_humidity', 'Plant5_monoxide', 'Plant5_dioxide', 'Plant5_pm2d5', 'Plant5_pm10', 'Plant5_temperature', 'Plant5_humidity', 'Plant6_monoxide', 'Plant6_dioxide', 'Plant6_pm2d5', 'Plant6_pm10', 'Plant6_temperature', 'Plant6_humidity', 'Plant7_monoxide', 'Pla

In [ ]:
temp = df[[f'Plant{i}_temperature' for i in range(1,11)]]
temp.head()

,Plant1_temperature,Plant2_temperature,Plant3_temperature,Plant4_temperature,Plant5_temperature,Plant6_temperature,Plant7_temperature,Plant8_temperature,Plant9_temperature,Plant10_temperature
2023-01-01 08:00:00,31.28,26.15,26.16,25.83,25.84,25.83,25.70,26.31,23.09,23.36
2023-01-01 08:10:00,31.00,26.23,26.20,25.93,25.88,25.95,25.70,26.38,23.49,23.73
2023-01-01 08:20:00,30.51,26.28,26.30,26.02,25.90,25.98,25.72,26.48,23.60,23.91
2023-01-01 08:30:00,30.33,26.36,26.41,26.17,26.01,26.05,25.90,26.53,23.76,23.99
2023-01-01 08:40:00,30.17,26.53,26.50,26.30,26.08,26.10,26.03,26.65,23.94,24.16


In [ ]:
temp.columns

Index(['Plant1_temperature', 'Plant2_temperature', 'Plant3_temperature',
       'Plant4_temperature', 'Plant5_temperature', 'Plant6_temperature',
       'Plant7_temperature', 'Plant8_temperature', 'Plant9_temperature',
       'Plant10_temperature'],
      dtype='object')

# 인과관계

정상성 factor 인과관계

In [ ]:
max_lag = 8

feature2planttemp = {}
stat_name = stat_name
stat_data = [df[[stat_name[i]]] for i in range(len(stat_name))]
print(len(stat_data))
print('정상성을 띄는 factor 개수 : ',len(stat_data))#, len(stat_name))
# print('---------------------------------------------------------------------')
for o in range(0,len(temp.columns)):
    y = temp[[temp.columns[o]]]
    feature2planttemp[o+1] = []
    print(f'<<<<<Plant{o+1}_temperature>>>>>')
    for j in range(len(stat_data)):
        test = pd.concat([stat_data[j],np.log1p(y).diff()], axis=1).dropna()
        try:
          result = grangercausalitytests(test,maxlag= max_lag,verbose=False) # lag 8
          print(stat_name[j])
          k=0
          for i in range(1,max_lag+1):
              # ssr F test & chi2 test , likelihood ratio test , parameter F test => 4개중 3개이상 유의수준보다 작으면 인과관계 있다고 판단.
              if sum([result[i][0]['ssr_ftest'][1]<0.05, result[i][0]['ssr_chi2test'][1]<0.05, result[i][0]['lrtest'][1]<0.05, result[i][0]['params_ftest'][1]<0.05])>=4:
                  k+=1
          if k==max_lag:
              # print(f'{j}는 인과성이 있다.')
              feature2planttemp[o+1].append(stat_name[j])
          # print()
        except: pass
# print('---------------------------------------------------------------------')
feature2planttemp

6
정상성을 띄는 factor 개수 :  6
<<<<<Plant1_temperature>>>>>
Plant2_air
Plant3_air
Plant3_dioxide
Plant5_air
Plant6_air
Plant7_air
<<<<<Plant2_temperature>>>>>
Plant2_air
Plant3_air
Plant3_dioxide
Plant5_air
Plant6_air
Plant7_air
<<<<<Plant3_temperature>>>>>
Plant2_air
Plant3_air
Plant3_dioxide
Plant5_air
Plant6_air
Plant7_air
<<<<<Plant4_temperature>>>>>
Plant2_air
Plant3_air
Plant3_dioxide
Plant5_air
Plant6_air
Plant7_air
<<<<<Plant5_temperature>>>>>
Plant2_air
Plant3_air
Plant3_dioxide
Plant5_air
Plant6_air
Plant7_air
<<<<<Plant6_temperature>>>>>
Plant2_air
Plant3_air
Plant3_dioxide
Plant5_air
Plant6_air
Plant7_air
<<<<<Plant7_temperature>>>>>
Plant2_air
Plant3_air
Plant3_dioxide
Plant5_air
Plant6_air
Plant7_air
<<<<<Plant8_temperature>>>>>
Plant2_air
Plant3_air
Plant3_dioxide
Plant5_air
Plant6_air
Plant7_air
<<<<<Plant9_temperature>>>>>
Plant2_air
Plant3_air
Plant3_dioxide
Plant5_air
Plant6_air
Plant7_air
<<<<<Plant10_temperature>>>>>
Plant2_air
Plant3_air
Plant3_dioxide
Plant5_air
Plant6

{1: ['Plant2_air',
  'Plant3_air',
  'Plant3_dioxide',
  'Plant5_air',
  'Plant6_air',
  'Plant7_air'],
 2: ['Plant3_air', 'Plant3_dioxide', 'Plant5_air', 'Plant6_air', 'Plant7_air'],
 3: ['Plant2_air',
  'Plant3_air',
  'Plant3_dioxide',
  'Plant5_air',
  'Plant6_air',
  'Plant7_air'],
 4: ['Plant2_air',
  'Plant3_air',
  'Plant3_dioxide',
  'Plant5_air',
  'Plant6_air',
  'Plant7_air'],
 5: ['Plant2_air', 'Plant3_dioxide', 'Plant5_air', 'Plant6_air', 'Plant7_air'],
 6: ['Plant2_air',
  'Plant3_air',
  'Plant3_dioxide',
  'Plant5_air',
  'Plant6_air',
  'Plant7_air'],
 7: ['Plant2_air',
  'Plant3_air',
  'Plant3_dioxide',
  'Plant5_air',
  'Plant6_air',
  'Plant7_air'],
 8: ['Plant2_air',
  'Plant3_air',
  'Plant3_dioxide',
  'Plant5_air',
  'Plant6_air',
  'Plant7_air'],
 9: ['Plant2_air',
  'Plant3_air',
  'Plant3_dioxide',
  'Plant5_air',
  'Plant6_air',
  'Plant7_air'],
 10: ['Plant3_air',
  'Plant3_dioxide',
  'Plant5_air',
  'Plant6_air',
  'Plant7_air']}

1차분 정상성 factor 인과관계

In [ ]:
max_lag = 8

diff1_name = diff1_name
diff1_data = [df[[diff1_name[i]]] for i in range(len(diff1_name))]
print(len(diff1_data))
print('정상성을 띄는 factor 개수 : ',len(diff1_data))#, len(diff1_name))
# print('---------------------------------------------------------------------')
for o in range(0,len(temp.columns)):
    y = temp[[temp.columns[o]]]

    print(f'<<<<<Plant{o+1}_temperature>>>>>')
    for j in range(len(diff1_data)):
        test = pd.concat([diff1_data[j],np.log1p(y).diff()], axis=1).dropna()
        try:
          result = grangercausalitytests(test,maxlag= max_lag,verbose=False) # lag 8
          print(diff1_name[j])
          k=0
          for i in range(1,max_lag+1):
              # ssr F test & chi2 test , likelihood ratio test , parameter F test => 4개중 3개이상 유의수준보다 작으면 인과관계 있다고 판단.
              if sum([result[i][0]['ssr_ftest'][1]<0.05, result[i][0]['ssr_chi2test'][1]<0.05, result[i][0]['lrtest'][1]<0.05, result[i][0]['params_ftest'][1]<0.05])>=4:
                  k+=1
          if k==max_lag:
              # print(f'{j}는 인과성이 있다.')
              feature2planttemp[o+1].append(diff1_name[j])
          # print()
        except: pass
# print('---------------------------------------------------------------------')
feature2planttemp

y --> 정상성 factor 인과관계

In [ ]:
max_lag = 8

planttemp1feature = {}
stat_name = stat_name
stat_data = [df[[stat_name[i]]] for i in range(len(stat_name))]
print(len(stat_data))
print('정상성을 띄는 factor 개수 : ',len(stat_data))#, len(stat_name))
# print('---------------------------------------------------------------------')
for o in range(0,len(temp.columns)):
    y = temp[[temp.columns[o]]]
    planttemp1feature[o+1] = []
    print(f'<<<<<Plant{o+1}_temperature>>>>>')
    for j in range(len(stat_data)):
        test = pd.concat([np.log1p(y).diff(),stat_data[j]], axis=1).dropna()
        try:
          result = grangercausalitytests(test,maxlag= max_lag,verbose=False) # lag 8
          print(stat_name[j])
          k=0
          for i in range(1,max_lag+1):
              # ssr F test & chi2 test , likelihood ratio test , parameter F test => 4개중 3개이상 유의수준보다 작으면 인과관계 있다고 판단.
              if sum([result[i][0]['ssr_ftest'][1]<0.05, result[i][0]['ssr_chi2test'][1]<0.05, result[i][0]['lrtest'][1]<0.05, result[i][0]['params_ftest'][1]<0.05])>=4:
                  k+=1
          if k==max_lag:
              # print(f'{j}는 인과성이 있다.')
              planttemp1feature[o+1].append(stat_name[j])
          # print()
        except: pass
# print('---------------------------------------------------------------------')
planttemp1feature

y --> 1차분 정상성 factor 인과관계

In [ ]:
max_lag = 8

diff1_name = diff1_name
diff1_data = [df[[diff1_name[i]]] for i in range(len(diff1_name))]
print(len(diff1_data))
print('정상성을 띄는 factor 개수 : ',len(diff1_data))#, len(diff1_name))
# print('---------------------------------------------------------------------')
for o in range(0,len(temp.columns)):
    y = temp[[temp.columns[o]]]

    print(f'<<<<<Plant{o+1}_temperature>>>>>')
    for j in range(len(diff1_data)):
        test = pd.concat([np.log1p(y).diff(),diff1_data[j]], axis=1).dropna()
        try:
          result = grangercausalitytests(test,maxlag= max_lag,verbose=False) # lag 8
          print(diff1_name[j])
          k=0
          for i in range(1,max_lag+1):
              # ssr F test & chi2 test , likelihood ratio test , parameter F test => 4개중 3개이상 유의수준보다 작으면 인과관계 있다고 판단.
              if sum([result[i][0]['ssr_ftest'][1]<0.05, result[i][0]['ssr_chi2test'][1]<0.05, result[i][0]['lrtest'][1]<0.05, result[i][0]['params_ftest'][1]<0.05])>=4:
                  k+=1
          if k==max_lag:
              # print(f'{j}는 인과성이 있다.')
              planttemp1feature[o+1].append(diff1_name[j])
          # print()
        except: pass
# print('---------------------------------------------------------------------')
planttemp1feature

In [ ]:
dict1 = feature2planttemp
dict2 = planttemp1feature

In [ ]:
dict1

{1: ['Plant2_air',
  'Plant3_air',
  'Plant3_dioxide',
  'Plant5_air',
  'Plant6_air',
  'Plant7_air',
  'Ahu1_inv',
  'Ahu2_inv',
  'Ahu3_inv',
  'Ahu4_inv',
  'Ahu5_inv',
  'Ahu6_inv',
  'Ahu7_inv',
  'Ahu8_inv',
  'Ahu9_inv',
  'Ahu10_inv',
  'Ahu11_inv',
  'Ahu12_inv',
  'Ahu13_inv',
  'Ahu14_inv',
  'Chiller1_inv',
  'Chiller2_inv',
  'Plant1_air',
  'Plant1_monoxide',
  'Plant1_pm2d5',
  'Plant1_pm10',
  'Plant1_temperature',
  'Plant1_humidity',
  'Plant2_dioxide',
  'Plant2_pm2d5',
  'Plant2_pm10',
  'Plant2_temperature',
  'Plant2_humidity',
  'Plant3_pm2d5',
  'Plant3_pm10',
  'Plant3_temperature',
  'Plant3_humidity',
  'Plant4_air',
  'Plant4_monoxide',
  'Plant4_dioxide',
  'Plant4_temperature',
  'Plant4_humidity',
  'Plant5_monoxide',
  'Plant5_dioxide',
  'Plant5_temperature',
  'Plant6_dioxide',
  'Plant6_temperature',
  'Plant7_dioxide',
  'Plant7_temperature',
  'Plant7_humidity',
  'Plant8_air',
  'Plant8_monoxide',
  'Plant8_dioxide',
  'Plant8_temperature',
  'Pla

In [ ]:
combined_dict = {}

for key in set(dict1) & set(dict2): # 교집합만
    combined_list = []

    if key in dict1:
        combined_list.extend(dict1[key])

    if key in dict2:
        combined_list.extend(dict2[key])

    combined_dict[key] = list(set(combined_list))

In [ ]:
combined_dict

{1: ['Plant6_dioxide',
  'Ahu2_inv',
  'Chiller2_inv',
  'Plant7_humidity',
  'Plant5_temperature',
  'Plant4_air',
  'Plant4_humidity',
  'Plant3_pm10',
  'Plant3_humidity',
  'Plant7_temperature',
  'Ahu6_inv',
  'Plant7_air',
  'Plant1_humidity',
  'Plant3_pm2d5',
  'Plant1_dioxide',
  'Plant8_pm2d5',
  'Plant3_dioxide',
  'Plant4_monoxide',
  'Plant2_air',
  'Plant3_air',
  'Plant2_dioxide',
  'Plant5_dioxide',
  'Plant1_monoxide',
  'Ahu14_inv',
  'Plant6_temperature',
  'Plant3_temperature',
  'Plant10_humidity',
  'Plant5_air',
  'Ahu4_inv',
  'Plant8_dioxide',
  'Plant8_air',
  'Plant9_temperature',
  'Plant1_air',
  'Plant8_pm10',
  'Plant6_air',
  'Ahu13_inv',
  'Plant8_humidity',
  'Plant4_dioxide',
  'Ahu7_inv',
  'Ahu3_inv',
  'Plant2_pm10',
  'Plant2_pm2d5',
  'Plant1_pm10',
  'Plant7_dioxide',
  'Plant5_monoxide',
  'Plant4_temperature',
  'Plant1_pm2d5',
  'Plant2_temperature',
  'Plant8_monoxide',
  'Plant8_temperature',
  'Plant9_humidity',
  'Ahu1_inv',
  'Ahu8_inv',

In [ ]:
# with open('result/feature2planttemp.pkl', 'wb') as handle:
#     pickle.dump(feature2planttemp, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 상관관계

In [ ]:
def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation.
    Shifted data filled with NaNs

    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length
    wrap : NaN 채우는 것. shift 하면서 사라진 값으로 다시 채우기. 값이 순환되게 된다. wrap=False 이면 NaN은 drop하고 correlation 구한다.
    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else:
        return datax.corr(datay.shift(lag))

In [ ]:
y = df[list(temp.columns)]
y.columns[0]

'Plant1_temperature'

In [ ]:
final_selection = {}
threshold = 0.55

combined_dict = dict1

for j in range(1,len(combined_dict)):
    select_name = combined_dict[j+1]
    select_data = [df[[combined_dict[j+1][k]]] for k in range(len(combined_dict[j+1]))]
    print(f'<<<<<plant{j+1}_temperature => {len(select_data)}>>>>>')
    final_selection[j+1] = []
    for i in range(len(select_data)):
        lags = 250
        try:
          rs = [crosscorr(select_data[i][select_data[i].columns[0]],y[y.columns[j]], lag) for lag in range(-lags-1,lags+1)]
        except:
          rs = [crosscorr(select_data[i],y[y.columns[0]], lag) for lag in range(-lags-1,lags+1)]
        offset = np.floor(len(rs)/2)-np.argmax(np.abs(rs)) # 최대 correlation 값 가지는 offset 계산 >> 절댓값
        if max(np.abs(rs)) >= threshold:
            final_selection[j+1].append(select_name[i])
            # print(select_name[i], max(np.abs(rs)))
    print(final_selection[j+1])

<<<<<plant2_temperature => 58>>>>>
['Ahu1_inv', 'Ahu2_inv', 'Ahu3_inv', 'Ahu4_inv', 'Ahu5_inv', 'Ahu7_inv', 'Ahu8_inv', 'Ahu9_inv', 'Ahu10_inv', 'Ahu11_inv', 'Ahu12_inv', 'Ahu14_inv', 'Plant1_temperature', 'Plant2_temperature', 'Plant2_humidity', 'Plant3_temperature', 'Plant3_humidity', 'Plant4_temperature', 'Plant5_temperature', 'Plant6_temperature', 'Plant7_temperature', 'Plant7_humidity', 'Plant8_temperature', 'Plant10_temperature']
<<<<<plant3_temperature => 58>>>>>
['Plant1_temperature', 'Plant2_temperature', 'Plant2_humidity', 'Plant3_temperature', 'Plant4_temperature', 'Plant5_temperature', 'Plant6_temperature', 'Plant7_temperature', 'Plant8_temperature']
<<<<<plant4_temperature => 58>>>>>
['Plant1_temperature', 'Plant2_temperature', 'Plant2_humidity', 'Plant3_temperature', 'Plant4_temperature', 'Plant5_temperature', 'Plant6_temperature', 'Plant7_temperature', 'Plant8_temperature']
<<<<<plant5_temperature => 58>>>>>
['Ahu1_inv', 'Ahu3_inv', 'Ahu8_inv', 'Ahu9_inv', 'Ahu10_inv', '

In [ ]:
final_selection

{1: ['Plant2_temperature',
  'Plant3_temperature',
  'Plant4_temperature',
  'Plant5_temperature',
  'Plant6_temperature',
  'Plant7_temperature',
  'Plant8_temperature',
  'Plant9_temperature',
  'Plant10_temperature'],
 2: ['Plant2_temperature',
  'Plant3_temperature',
  'Plant4_temperature',
  'Plant5_temperature',
  'Plant6_temperature',
  'Plant7_temperature',
  'Plant8_temperature',
  'Plant10_temperature'],
 3: ['Plant2_temperature',
  'Plant3_temperature',
  'Plant4_temperature',
  'Plant5_temperature',
  'Plant6_temperature',
  'Plant7_temperature',
  'Plant8_temperature'],
 4: ['Plant2_temperature',
  'Plant3_temperature',
  'Plant4_temperature',
  'Plant5_temperature',
  'Plant6_temperature',
  'Plant7_temperature',
  'Plant8_temperature'],
 5: ['Plant2_temperature',
  'Plant3_temperature',
  'Plant4_temperature',
  'Plant5_temperature',
  'Plant6_temperature',
  'Plant7_temperature',
  'Plant8_temperature',
  'Plant10_temperature'],
 6: ['Plant2_temperature',
  'Plant3_temp

In [ ]:
# with open('result/final_selection.pkl', 'wb') as handle:
#     pickle.dump(final_selection, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('result/plant_temperature.pkl', 'wb') as handle:
#     pickle.dump(temp, handle, protocol=pickle.HIGHEST_PROTOCOL)

# VAR

In [ ]:
final_selection[1]

In [ ]:
temp.head()

,Plant1_temperature,Plant2_temperature,Plant3_temperature,Plant4_temperature,Plant5_temperature,Plant6_temperature,Plant7_temperature,Plant8_temperature,Plant9_temperature,Plant10_temperature
2023-01-31 10:00:00,27.26,25.84,26.13,26.04,25.81,26.22,25.86,26.84,24.43,24.88
2023-01-31 10:10:00,26.48,25.88,25.96,25.96,25.64,26.15,25.79,26.84,24.77,25.26
2023-01-31 10:20:00,26.39,25.75,25.80,25.77,25.45,26.02,25.60,26.74,25.06,25.57
2023-01-31 10:30:00,25.87,25.63,25.72,25.60,25.27,25.89,25.46,26.65,25.35,25.72
2023-01-31 10:40:00,25.43,25.63,25.66,25.53,25.24,25.90,25.34,26.58,25.66,26.01


In [ ]:
df_var = pd.DataFrame(columns=final_selection[1], index=df.index)
for i in range(len(final_selection[1])):
    if final_selection[1][i] in stat_name: # 정상성 띄면 그대로, 안띄면 1차차분처리
        df_var[final_selection[1][i]] = df[final_selection[1][i]]
    else: df_var[final_selection[1][i]] = np.log1p(df[final_selection[1][i]]).diff()
df_var[temp.columns[0]] = np.log1p(temp[temp.columns[0]]).diff() # temperature는 1차분해야 정상성
df_var.dropna(inplace=True)
df_var.tail()

,Plant7_dioxide,Ahu7_inv,Plant9_humidity,Ahu5_inv,Ahu11_inv,Plant10_temperature,Ahu4_inv,Plant9_temperature,Ahu9_inv,Plant2_temperature,Ahu3_inv,Ahu1_inv,Ahu13_inv,Plant1_temperature
2023-08-01 15:20:00,0.020907,0.0,0.007813,0.0,0.0,-0.013522,0.0,-0.016504,-0.102654,0.000367,0.0,0.0,0.0,-0.002010
2023-08-01 15:30:00,-0.013889,0.0,-0.001812,0.0,0.0,-0.002789,0.0,0.000000,0.000000,-0.005524,0.0,0.0,0.0,0.000000
2023-08-01 15:40:00,0.010435,0.0,0.007227,0.0,0.0,0.001860,0.0,-0.003086,0.000000,0.002950,0.0,0.0,0.0,-0.003628
2023-08-01 15:50:00,0.006897,0.0,0.002876,0.0,0.0,-0.004345,0.0,0.000000,0.000000,-0.001474,0.0,0.0,0.0,-0.002021
2023-08-01 16:00:00,0.013652,0.0,-0.005219,0.0,0.0,0.009904,0.0,0.004318,0.102654,0.002578,0.0,0.0,0.0,0.001213


In [ ]:
var_naive = VAR(df_var)
order_results = var_naive.select_order(maxlags=30)
# min(order_results.aic, key=order_results.aic.get)
# var_naive.select_order(maxlags=50).summary() #27

1

In [ ]:
print(order_results.aic)
order_results.summary()

27


,AIC,BIC,FPE,HQIC
0,-73.76,-73.76,9.244e-33,-73.76
1,-75.20,-75.13,2.200e-33,-75.17
2,-75.75,-75.61,1.267e-33,-75.70
3,-76.27,-76.07,7.504e-34,-76.21
4,-76.43,-76.16,6.420e-34,-76.34
5,-76.55,-76.21,5.696e-34,-76.44
6,-76.70,-76.30,4.885e-34,-76.57
7,-76.77,-76.30,4.569e-34,-76.62
8,-76.90,-76.37,3.990e-34,-76.73
9,-76.97,-76.37*,3.723e-34,-76.78


In [ ]:
model_naive = var_naive.fit(maxlags = order_results.aic, trend = "c") #27
model_naive.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 07, Dec, 2023
Time:                     10:03:40
--------------------------------------------------------------------
No. of Equations:         14.0000    BIC:                   -75.5177
Nobs:                     23870.0    HQIC:                  -76.7312
Log likelihood:           453866.    FPE:                2.64868e-34
AIC:                     -77.3139    Det(Omega_mle):     2.12446e-34
--------------------------------------------------------------------
Results for equation Plant7_dioxide
                             coefficient       std. error           t-stat            prob
------------------------------------------------------------------------------------------
const                           0.000000         0.000063            0.006           0.995
L1.Plant7_dioxide              -0.090919         0.006546          -13.890           0.000
L1.Ahu7_inv

In [ ]:
f = pd.date_range(start='2023-08-01 16:10:00', end='2023-08-15 00:00:00', freq = '10T')
len(f)

1920

In [ ]:
df_var[df_var.columns[-1]]

2023-01-31 10:20:00   -0.003280
2023-01-31 10:30:00   -0.019168
2023-01-31 10:40:00   -0.016511
2023-01-31 10:50:00   -0.014098
2023-01-31 11:00:00   -0.009640
                         ...   
2023-08-01 15:20:00   -0.002010
2023-08-01 15:30:00    0.000000
2023-08-01 15:40:00   -0.003628
2023-08-01 15:50:00   -0.002021
2023-08-01 16:00:00    0.001213
Name: Plant1_temperature, Length: 23897, dtype: float64

In [ ]:
opt_input= df_var.values[-1*order_results.aic:] # optimal lag is 27
steps = pd.date_range(start='2023-08-01 16:10:00', end='2023-08-15 00:00:00', freq = '10T')
forecast_naive = model_naive.forecast(y = opt_input, steps = len(steps))
df_forecast = pd.DataFrame(forecast_naive, columns = df_var.columns)
df_forecast.index = steps
df_forecast[df_forecast.columns[-1]] = np.exp(np.r_[np.log1p(temp[temp.columns[0]]).iloc[-1], df_forecast[df_forecast.columns[-1]]].cumsum()[1:])-1
df_forecast

,Plant7_dioxide,Ahu7_inv,Plant9_humidity,Ahu5_inv,Ahu11_inv,Plant10_temperature,Ahu4_inv,Plant9_temperature,Ahu9_inv,Plant2_temperature,Ahu3_inv,Ahu1_inv,Ahu13_inv,Plant1_temperature
2023-08-01 16:10:00,0.001314,-0.127209,-0.004674,-0.125890,-0.092546,0.002159,-0.085644,0.002767,-0.140244,-0.000247,-0.173009,-0.210034,-0.132170,23.753918
2023-08-01 16:20:00,0.004527,-0.096361,0.002529,-0.017894,-0.119980,-0.001987,-0.114244,-0.001539,-0.059508,-0.000335,-0.004145,-0.023362,-0.118058,23.779710
2023-08-01 16:30:00,0.002958,-0.153269,0.006106,-0.118666,-0.126233,-0.002547,-0.030773,-0.002082,-0.157077,0.000118,-0.066879,-0.094943,-0.138475,23.850812
2023-08-01 16:40:00,0.003230,-0.137979,0.004169,-0.075692,-0.085476,-0.001867,-0.103685,-0.001691,-0.007705,0.000260,-0.092421,-0.077068,-0.061181,23.945758
2023-08-01 16:50:00,0.002959,-0.119460,0.004106,-0.197255,-0.111610,-0.002049,-0.083209,-0.001846,-0.170087,0.000289,-0.112488,-0.114913,-0.089360,24.024518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-14 23:20:00,0.000003,0.000084,0.000017,-0.000026,-0.000019,0.000009,0.000038,0.000009,0.000079,0.000011,-0.000002,0.000093,0.000077,25.917847
2023-08-14 23:30:00,0.000003,0.000084,0.000017,-0.000026,-0.000019,0.000009,0.000038,0.000009,0.000079,0.000011,-0.000002,0.000093,0.000077,25.917986
2023-08-14 23:40:00,0.000003,0.000084,0.000017,-0.000026,-0.000019,0.000009,0.000038,0.000009,0.000079,0.000011,-0.000002,0.000093,0.000077,25.918125
2023-08-14 23:50:00,0.000003,0.000084,0.000017,-0.000026,-0.000019,0.000009,0.000038,0.000009,0.000079,0.000011,-0.000002,0.000093,0.000077,25.918263
